# Проект: обработка комментариев

## Подготовка

In [72]:
import pandas as pd
import numpy as np

import re
import spacy
import nltk
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score

In [74]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [75]:
data = data.rename(columns={'Unnamed: 0':'index'})
data = data.set_index('index')

In [76]:
data.duplicated(subset=['text']).sum()

0

Дубликатов нет

Сделаем выборку, чтобы не обучаться слишком долго

In [77]:
sample = data.sample(n=8000, random_state=12345).reset_index(drop=True)

In [ ]:
m = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def lemmatize(text):
    return "".join(str(m(text)))

In [79]:
m = spacy.load('en_core_web_sm')
def lemmatize(text):
    return " ".join([token.lemma_ for token in m(text)])

In [80]:
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z]', ' ', text).lower().split()).replace('\n',' ')

In [81]:
sample['lemm'] = sample['text'].apply(
    lambda x: lemmatize(clear_text(x)))

In [83]:
#nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

In [84]:
sample['toxic'].value_counts()

0    7182
1     818
Name: toxic, dtype: int64

In [85]:
corpus_train, corpus_test, target_train, target_test = train_test_split(
    sample['lemm'], sample['toxic'], test_size=0.1, random_state=12345)

In [86]:
corpus_train, corpus_valid, target_train, target_valid = train_test_split(
    corpus_train, target_train, test_size=0.11, random_state=12345)

In [87]:
target_train.value_counts()

0    5782
1     626
Name: toxic, dtype: int64

In [88]:
target_valid.value_counts()

0    706
1     86
Name: toxic, dtype: int64

In [89]:
target_test.value_counts()

0    694
1    106
Name: toxic, dtype: int64

In [90]:
corpus_train = corpus_train.values
corpus_test = corpus_test.values
corpus_valid = corpus_valid.values

In [91]:
vectorizer = TfidfVectorizer(stop_words=stopwords, analyzer='word', ngram_range=(1,1))
features_train = vectorizer.fit_transform(corpus_train)
features_valid = vectorizer.transform(corpus_valid)
features_test = vectorizer.transform(corpus_test)

## Обучение моделей

In [92]:
bst=0
scr=0
for i in np.arange(0.1,0.9,0.05):
    model_lr = LogisticRegression(max_iter=1000, solver='liblinear', class_weight='balanced',C=i, penalty='l2')
    model_lr.fit(features_train,target_train)
    s = f1_score(model_lr.predict(features_valid),target_valid)
    if s>scr:
        scr=s
        bst=i
print(bst,scr)

0.20000000000000004 0.7471264367816092


In [99]:
model_lr = LogisticRegression(max_iter=1000, solver='liblinear', class_weight='balanced',C=0.2, penalty='l2')
model_lr.fit(features_train,target_train);

In [94]:
model_cb = CatBoostClassifier(loss_function="Logloss", iterations=100 ,learning_rate=0.1)
model_cb.fit(features_train, target_train, verbose=100)
f1_score(model_cb.predict(features_valid),target_valid)

0:	learn: 0.6096746	total: 375ms	remaining: 37.1s
99:	learn: 0.1425397	total: 19.8s	remaining: 0us


0.609375

## Тестирование

In [98]:
f1_score(model_lr.predict(features_test),target_test)

0.7735849056603775

In [96]:
f1_score(model_cb.predict(features_test),target_test)

0.6455696202531646

## Выводы

В данной работе была решена задача распознавания токсичных комментариев

В данных нет дубликатов.

Для обучения модели была сделана выборка из 8000 значений. Она разбита на тренировочную, валидационную и тестовую выборку.

Для создания признаков были исползованы лемматизация, регулярные выражения, TfidfVectorizer.

Выбраны модели LogisticRegression и CatBoostClassifier

Метрика f1 LogisticRegression на валидационной выборке: 0.75

Метрика f1 CatBoostClassifier на валидационной выборке: 0.61

Метрика f1 LogisticRegression на тестовой выборке: 0.77

Метрика f1 CatBoostClassifier на тестовой выборке: 0.65

Модель LogisticRegression достигла требуемого значения на тестовой выборке